<a href="https://colab.research.google.com/github/tayfununal/Normalizing-Flows/blob/main/ode_new_with_mse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp

plt.style.use('seaborn-poster')

%matplotlib inline
"""
def F(t,s):       # Diff eq F(t,s) = 0
  return [np.cos(t), -np.sin(t)]
"""
def F(t,y):       # Diff eq F(t,s) = 0
  return [y[1], -y[0]]

t_eval = np.arange(0, 2*np.pi, 0.01)
sol = solve_ivp(F, [0, 2*np.pi], [0., 1.], t_eval=t_eval, dense_output=True)


plt.figure(figsize = (8, 8))

plt.subplot(221)
plt.plot(sol.t, sol.y[0])
plt.xlabel('t')
plt.ylabel('H[0] = sin(t)')

plt.subplot(222)
plt.plot(sol.t, sol.y[0] - np.sin(sol.t))
plt.xlabel('t')
plt.ylabel('H[0] - sin(t)')

plt.subplot(223)
plt.plot(sol.t, sol.y[1])
plt.xlabel('t')
plt.ylabel('H[1] = cos(t)')

plt.subplot(224)
plt.plot(sol.t, sol.y[1] - np.cos(sol.t))
plt.xlabel('t')
plt.ylabel('H[1] - cos(t)')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (4, 4))
plt.plot(sol.y.T[:, 0], sol.y.T[:, 1])
plt.xlabel('q')
plt.ylabel('p')
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

In [ ]:
y = sol.y.T # etiketlerimiz
t = sol.t   # t değerlerimiz

In [ ]:
t = t.reshape(([t.shape[0],1]))
shuffle = np.concatenate((y,t), axis=1)
np.random.shuffle(shuffle)
shuffle

In [ ]:
y = shuffle[:,:2]
t = shuffle[:,2]
print('y:',y, '\nt:',t)

In [ ]:
"""tf.random.set_seed(39)
np.random.seed(39)"""

girdi = Input(shape=(1,))

hidden = Dense(512, activation= 'tanh')(girdi)
hidden = Dense(512, activation= 'tanh')(hidden)
hidden = Dense(2)(hidden)


model = Model(inputs=girdi, outputs=hidden)

In [ ]:
model.compile(optimizer='adam', loss='mse')
model.fit(x=t, y= y, epochs=100, batch_size=16)

In [ ]:
t_deneme = np.arange(0, 2*np.pi, 0.02).reshape((315,1)) #629
pred = model.predict(t_deneme)

plt.figure(figsize = (4, 4))
plt.plot(pred[:, 0], pred[:, 1])
plt.plot(sol.y.T[:, 0], sol.y.T[:, 1])
plt.xlabel('q')
plt.ylabel('p')
plt.show()

In [ ]:
def H(z):
  return tf.reduce_sum((z**2)/2,axis=1)
  
hamiltonian = H(pred)
true_hamiltonian = H(sol.y.T)

plt.figure(figsize = (4, 4))
plt.plot(t_deneme,hamiltonian)
plt.plot(t_eval, true_hamiltonian)
plt.xlabel('t')
plt.ylabel('H')
plt.show()